In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from numba import jit
from datetime import datetime
import multiprocessing

In [ ]:
sigma = 0.15

Spectrum_data = np.loadtxt('spec3.7.dat')

Xdata = Spectrum_data[:,0]
Ydata = Spectrum_data[:,1]
Xmin = Xdata[0]
Xmax = Xdata[-1]

In [ ]:
@jit
def gauss(x, mean, sigma):
    '''This function calculates normalized gaussian probability distribution'''
    arg = (x-mean)/sigma
    res = np.exp(-0.5 * (arg**2))     # taking the square of the arg and the multilying with (-0.5) and then calculating the exponential
    norm_res = res / (np.sqrt(2*np.pi) * sigma)   # normalizing the result by dividing the res with (sqrt(2*pi) * sigma)
    return norm_res

In [ ]:
EvaluationFunction = interpolate.interp1d(Xdata, Ydata)

def evaluate(x, Xboundary_min=Xmin, Xboundary_max=Xmax):
    '''Given a value of x, this function will evaluate y-value. If x is within boundary then will
    return the corresponding y-value calculated from evaluation function. Esle will return 0.
    This is required because evaluation function can only evaluate y-value if x is within the 
    boundary of Xdata. Otherwise this gives error'''
    if Xboundary_min <= x <= Xboundary_max:
        return EvaluationFunction(x)
    else:
        return 0

In [ ]:

def SmearingFunction(x, sigma=sigma, Xboundary_min=Xmin, Xboundary_max=Xmax):
    '''This function does gaussian smearing of spectrum.
       @param x: x-point at which gaussian smearing needs to be done
       @param sigma: sigma value of the gaussian kernel
       @param Xboundary_min: lower x boundary of the spectrum. Default value is Xmin
       @param Xboundary_max: upper x boundary of the spectrum. Default value is Xmax
       @return: returns smeared y-value
    '''
    # The smeared spectrum is F(x) = Integral of [ Gauss(t) * f(x-t) ] dt
    # Here f(x-t) is the not-Smeared spectrum
    # We will integrate in the limit from ( -3*sigma to +3*sigma )
    # Step size of Integration will be 0.001*sigma which we define as Mesh
    
    Mesh = 0.001 * sigma
    ThisY = 0.0
    SumY = 0.0
    SumGauss = 0.0
    
    # To make distinction what is x and where we do the Integration. We integrate at X_evaluate ---------
    X_evaluate = x
    # we need this, if the x is near zero then the function will collapse ---------
    if (x < 2*sigma):           
        X_evaluate = 2*sigma
    
    # Integrate in the limit from ( -3*sigma to +3*sigma ). Here t is the variable of integral. ---------
    for t in np.arange(start=-3.0*sigma, stop=3.0*sigma, step=Mesh):
        ThisY = gauss(t, 0.0, sigma)                         # calculate gaussian at t position
        ThisY = ThisY * evaluate(X_evaluate - t)             # Gauss(t) * f(x-t); f(x-t): not-smeared_Counts(x-t) is calculated from evaluate function
        SumY = SumY + ThisY                                  # do the summation
        
        # If (X_evaluate - t) is outside Xboundary_max then do Gaussian integral in this way
        if ((X_evaluate - t) > Xboundary_max):
            SumGauss = SumGauss + gauss(t, 0.0, sigma)
    
    # Multiply with Mesh to finish the Integral ----------
    Integral = SumY * Mesh
    GaussIntegral = SumGauss * Mesh

     # Correct for the missing Gaussian surface near the end. Applicable only 
     # when (X_evaluate - t) is outside Xboundary_max. Otherwise GaussIntegral is zero
    Smeared = Integral/(1.0 - GaussIntegral)
    
    # this is applicable for x >= 2.0*Sigma
    Answer = Smeared
    
    # But if x is in between 0 and 2.0*sigma then calculate smeared value in this way
    if (0.0 <= x < 2.0*sigma):
        Ratio = Smeared / evaluate(2.0*sigma)
        Value = evaluate(x)
        Answer = Value * Ratio
    
    # And if x < 0.0, then
    if (x < 0.0):
        Answer = 0.0
    
     # Calculation done and return Answer
    return Answer

# SmearingFunction(10)

In [ ]:
if __name__ == "__main__":
    
    eenergy = Xdata
    counts = Ydata
    smeared_count = []
    
    print("Spectrum Smearing has started")
    timestamp_1 = datetime.now()
    
    for ex_energy in eenergy:
        smeared_count.append(SmearingFunction(ex_energy))
        
#     print(smeared_count)

    timestamp_2 = datetime.now()
    Tot_time = timestamp_2 - timestamp_1
    print("Smearing Finished!")
    print("Total time taken to smear the spectrum:", Tot_time)

    

In [ ]:
if __name__ == "__main__":
    
    timestamp_start = datetime.now()
    eenergy = Xdata
    counts = Ydata
    smeared_count = []
    
    print("Spectrum Smearing has started")
    
    np = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=np)
    
    print("Running across %d Cores"%np)
    
    smeared_count = pool.map(SmearingFunction, eenergy)

    timestamp_finish = datetime.now()
    Tot_time = timestamp_finish - timestamp_start
    print("Smearing Finished!")
    print("Total time taken to smear the spectrum:", Tot_time)

    